In [4]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup as BS
from konlpy.tag import Mecab
import pandas as pd

In [5]:
mecab = Mecab()

In [6]:
today = datetime.today().strftime("%Y%m%d")

In [7]:
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

In [8]:
def get_news_links(page):
    url = base_url + "&page=" + str(page)
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

def get_news_content(url):
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article',{'id':'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

In [9]:
# 크롤링 시작
news_links = []
page = 1

while True:
    links = get_news_links(page)
    if not links or any(link in news_links for link in links):
        break
    news_links.extend(links)
    page += 1


In [12]:

# 뉴스 기사 내용 크롤링
news_contents = []
for link in news_links[:100]:
    try:
        title, content = get_news_content(link)
        news_contents.append((title, content))
    except Exception as e:
        print(f"Failed to get content from {link}: {e}")

In [10]:
news_links[:100]

['https://n.news.naver.com/mnews/article/014/0005204961?sid=101',
 'https://n.news.naver.com/mnews/article/014/0005204961?sid=101',
 'https://n.news.naver.com/mnews/article/005/0001706599?sid=101',
 'https://n.news.naver.com/mnews/article/005/0001706599?sid=101',
 'https://n.news.naver.com/mnews/article/144/0000971385?sid=101',
 'https://n.news.naver.com/mnews/article/144/0000971385?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573009?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573008?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573008?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573007?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573006?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573006?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573005?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573004?sid=101',
 'https://n.news.naver.com/mnews/article/020/0003573004?sid=101',
 'https://

In [13]:
news_contents

[('"전 세계, 식량전쟁으로 치닫는다" 곡물메이저 올람아그리 경고',
  '[파이낸셜뉴스]   \n\n\n\n기후 위기와 지정학적 긴장 속에 전 세계가 \'식량 전쟁\'에 나설 것이라는 경고가 나왔다. 지난해 9월 8일(현지시간) 러시아 옴스크 지역에서 농민들이 밀을 수확하고 있다. 로이터 뉴스1  전 세계가 지정학적 긴장과 기후 변화 속에 \'식량 전쟁\'으로 치닫고 있다고 세계 최대 곡물 메이저 가운데 한곳이 경고했다.   식량 공급이 감소하면서 각국이 식량 확보를 위해 치고받는 전쟁 상황을 향해 나아가고 있다는 것이다.   파이낸셜타임스(FT)는 26일(현지시간) 싱가포르에 본사가 있는 세계 최대 곡물 거래 업체 가운데 한 곳인 올람아그리 최고경영자(CEO) 서니 베르지스가 이같이 경고했다고 보도했다.   베르지스는 "인류는 석유를 놓고 수많은 전쟁을 치렀다"면서 "앞으로 식량과 물을 놓고 더 큰 전쟁들을 벌일 것"이라고 경고했다.   FT에 따르면 베르지스는 20일 로스차일드 산하의 레드번애틀랜틱이 주최한 소비자 컨퍼런스에서 각국이 식량 재고 확보를 위해 무역 장벽을 치면서 식료품 가격 인플레이션(물가상승)이 악화했다면서 이같이 경고했다.   베르지스의 이 같은 경고는 곡물 메이저들이 중간에 농간을 부려 막대한 차익을 내기 위해 상황을 악용하는 것이 아니냐는 비판이 고조되는 가운데 나왔다.   올람아그리를 비롯한 곡물 메이저들은 2022년 2월 러시아가 우크라이나를 전면 침공해 곡물 가격이 뛰기 시작하자 사상 최대 순익을 기록하고 있다.   이들이 공급을 통제하면서 전 세계 식료품 가격 폭등을 부르고 있다는 비판이 높다.   베르지스는 그러나 각국 정부의 시장 개입이 식료품 가격 고공행진 원인 가운데 하나라고 주장했다.   그는 2022년 우크라이나 전쟁 여파로 각국이 비관세 무역장벽을 쌓기 시작했다면서 무역장벽을 세운 나라가 154개국에서 1266개국으로 늘었다고 말했다.   베르지스는 이같은 비관세 무역장벽이 "수급 불균형 악화를 만들어냈다"고 주

In [14]:
len(news_contents)

100

In [15]:
#중복값 제거
set(news_contents)

{('"전 세계, 식량전쟁으로 치닫는다" 곡물메이저 올람아그리 경고',
  '[파이낸셜뉴스]   \n\n\n\n기후 위기와 지정학적 긴장 속에 전 세계가 \'식량 전쟁\'에 나설 것이라는 경고가 나왔다. 지난해 9월 8일(현지시간) 러시아 옴스크 지역에서 농민들이 밀을 수확하고 있다. 로이터 뉴스1  전 세계가 지정학적 긴장과 기후 변화 속에 \'식량 전쟁\'으로 치닫고 있다고 세계 최대 곡물 메이저 가운데 한곳이 경고했다.   식량 공급이 감소하면서 각국이 식량 확보를 위해 치고받는 전쟁 상황을 향해 나아가고 있다는 것이다.   파이낸셜타임스(FT)는 26일(현지시간) 싱가포르에 본사가 있는 세계 최대 곡물 거래 업체 가운데 한 곳인 올람아그리 최고경영자(CEO) 서니 베르지스가 이같이 경고했다고 보도했다.   베르지스는 "인류는 석유를 놓고 수많은 전쟁을 치렀다"면서 "앞으로 식량과 물을 놓고 더 큰 전쟁들을 벌일 것"이라고 경고했다.   FT에 따르면 베르지스는 20일 로스차일드 산하의 레드번애틀랜틱이 주최한 소비자 컨퍼런스에서 각국이 식량 재고 확보를 위해 무역 장벽을 치면서 식료품 가격 인플레이션(물가상승)이 악화했다면서 이같이 경고했다.   베르지스의 이 같은 경고는 곡물 메이저들이 중간에 농간을 부려 막대한 차익을 내기 위해 상황을 악용하는 것이 아니냐는 비판이 고조되는 가운데 나왔다.   올람아그리를 비롯한 곡물 메이저들은 2022년 2월 러시아가 우크라이나를 전면 침공해 곡물 가격이 뛰기 시작하자 사상 최대 순익을 기록하고 있다.   이들이 공급을 통제하면서 전 세계 식료품 가격 폭등을 부르고 있다는 비판이 높다.   베르지스는 그러나 각국 정부의 시장 개입이 식료품 가격 고공행진 원인 가운데 하나라고 주장했다.   그는 2022년 우크라이나 전쟁 여파로 각국이 비관세 무역장벽을 쌓기 시작했다면서 무역장벽을 세운 나라가 154개국에서 1266개국으로 늘었다고 말했다.   베르지스는 이같은 비관세 무역장벽이 "수급 불균형 악화를 만들어냈다"고 주

In [16]:
len(set(news_contents))

59

In [17]:

# Create a DataFrame to store titles and nouns
df = pd.DataFrame(columns=["Title", "Nouns"])

# Text analysis using Mecab and store in DataFrame
for i, (title, content) in enumerate(set(news_contents), start=1):
    if content:
        nouns = mecab.nouns(content)
        new_row = pd.DataFrame({"Title": [title], "Nouns": [nouns]})
        df = pd.concat([df, new_row], ignore_index=True)        

# Display the DataFrame
print(df)

                                                Title  \
0                               “에어컨 사용 전 먼지필터 청소하세요”   
1                     전기차 개발 애먹는 유럽… “기술? 이젠 돈으로 사겠어”   
2                    [주식 매매 상위 종목 및 환율] 2024년 6월 26일자   
3                                “모듈러주택 개발 우리가 앞장섭니다”   
4                   [단신] 한국앤컴퍼니, ‘2024 산단 오픈이노베이션’ 참여   
5   韓, 주요 12개국 중 양자기술 꼴찌 그쳐… 美 100점 받은 양자컴퓨터, 2.3점 받아   
6                   자영업 취약 대출자 연체율 10% 넘어… PF 부실도 눈덩이   
7                            ‘롯데家 3세’ 신유열, 日롯데홀딩스 이사로   
8                      HJ중공업, 상반기 2조 수주…누적 잔고 7조 돌파했다   
9                                    은행권 이자순이익 34兆 돌파   
10                               우리금융, 동양-ABL생명 인수 추진   
11                         출연硏 예산-인건비 자율성 높이고 2년마다 평가   
12                 "전 세계, 식량전쟁으로 치닫는다" 곡물메이저 올람아그리 경고   
13                               강남·광화문 빗물 터널도 아직 미착공   
14                                           햇감자 할인판매   
15                       “지역에서 받은 사랑, 자립준비청년에게 돌려줍니다”   
16                             

In [18]:
df

,Title,Nouns
0,“에어컨 사용 전 먼지필터 청소하세요”,"[지난해, 서비스, 신청, 고장, 실외기, 주변, 정리, 뒤, 사용, 이달, 최고,..."
1,전기차 개발 애먹는 유럽… “기술? 이젠 돈으로 사겠어”,"[후, 발주자, 기술, 구매, 활로, 그래픽, 미드, 저니, 일, 현지, 시각, 독..."
2,[주식 매매 상위 종목 및 환율] 2024년 6월 26일자,"[주식, 매매, 상위, 종목, 환율, 년, 월, 일자]"
3,“모듈러주택 개발 우리가 앞장섭니다”,"[부산, 도시, 공사, 사내, 동아리, 결성, 매입, 임대, 주택, 등, 적용, 연..."
4,"[단신] 한국앤컴퍼니, ‘2024 산단 오픈이노베이션’ 참여","[한국, 앤, 컴퍼니, 산단, 오픈, 이노베이션, 참여, 한국, 앤, 컴퍼니, 산업..."
5,"韓, 주요 12개국 중 양자기술 꼴찌 그쳐… 美 100점 받은 양자컴퓨터, 2.3점 받아","[과기부, 미래, 대, 기술, 비교, 평가, 위, 바이오, 위, 한국, 양자, 기술..."
6,자영업 취약 대출자 연체율 10% 넘어… PF 부실도 눈덩이,"[고금리, 내수, 부진, 자영업, 연체, 상승, 취약, 대출자, 채무, 조정, 필요..."
7,"‘롯데家 3세’ 신유열, 日롯데홀딩스 이사로","[일본, 롯데, 홀딩스, 일, 정기, 주주, 총회, 신동빈, 롯데, 그룹, 회장, ..."
8,"HJ중공업, 상반기 2조 수주…누적 잔고 7조 돌파했다","[새만금, 공항, 등, 건설, 부문, 선전, 조선, 부문, 계약, 문, 증가, 수익..."
9,은행권 이자순이익 34兆 돌파,"[금리, 상승기, 기업, 대출, 급증, 영향, 한국은행, 일, 발표, 년, 상반기,..."


In [24]:
test=df.loc[0, 'Nouns']

In [25]:
len(test)

156

In [26]:
len(set(test))

89

In [27]:
from collections import Counter

In [30]:
# Counter 객체 생성
counter = Counter(test)

# 중복된 값과 개수를 저장할 리스트
data = []

# Counter 객체에서 중복된 값과 개수를 추출하여 리스트에 저장
for item, count in counter.items():
    if count > 1:
        data.append({'값': item, '개수': count})

# 데이터프레임 생성
duplicated_df = pd.DataFrame(data)


In [35]:
duplicated_df.sort_values(by="개수",ascending=False)

,값,개수
8,보험,9
12,금융,8
7,생명,7
11,우리,7
19,매각,5
9,인수,5
22,손해,4
0,은행,3
26,사,3
23,입찰,3
